In [58]:
#Habilitar si usas google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
import pandas as pd

# Lectura de la data

In [60]:
# Utilizando google colab Jesus
#data = pd.read_csv('/content/drive/MyDrive/IA/PracticaCalificada2-CC421/train.csv', chunksize=42000000)
# Utilizando google colab Franz
data = pd.read_csv('/content/drive/MyDrive/IA_PC2/train.csv', chunksize=1000000)
# Desde local (jupyter notebook)
#data = pd.read_csv('train.csv', chunksize=42000000)

In [61]:
data.get_chunk()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1
...,...,...,...,...,...,...,...,...
999995,2014-09-13 21:44:38.0000001,7.0,2014-09-13 21:44:38 UTC,-73.976676,40.785630,-73.959196,40.775114,1
999996,2010-09-20 14:50:37.0000002,7.3,2010-09-20 14:50:37 UTC,-73.992103,40.671385,-73.992103,40.671385,2
999997,2013-04-26 14:03:00.000000118,10.5,2013-04-26 14:03:00 UTC,-73.978423,40.751135,-73.993397,40.762197,5
999998,2011-07-08 00:29:00.00000099,6.9,2011-07-08 00:29:00 UTC,-73.980317,40.759482,-73.976832,40.743122,1


In [62]:
df = data.get_chunk() 

In [63]:
print(df.head())

                                   key  ...  passenger_count
1000000    2010-03-06 21:13:04.0000003  ...                1
1000001  2010-03-10 16:19:00.000000128  ...                1
1000002    2014-02-27 08:50:19.0000006  ...                1
1000003    2011-02-11 17:46:17.0000005  ...                3
1000004   2011-07-25 12:48:00.00000074  ...                1

[5 rows x 8 columns]


# Preprocesamiento de la data

Se cuentan las filas con valores nulos, como estás representan una cantidad infíma
simplemente se eliminarán

In [64]:
print(df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    4
dropoff_latitude     4
passenger_count      0
dtype: int64


In [65]:
df.dropna(inplace = True)
print(df.isnull().sum())

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    0
dropoff_latitude     0
passenger_count      0
dtype: int64


Se utilizará solo la hora del tiempo de recogo, para ello lo extraemos en otra columna

In [66]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], format='%Y-%m-%d %H:%M:%S UTC')

In [67]:
df['pickup_hour'] =df['pickup_datetime'].dt.hour

In [68]:
print(df['pickup_hour'])

1000000    21
1000001    16
1000002     8
1000003    17
1000004    12
           ..
1999995    13
1999996    17
1999997    22
1999998     9
1999999    15
Name: pickup_hour, Length: 999996, dtype: int64


En la columna de números de pasajeros los valores están entre 0-6, existen pocos valores mayores a 7, asi que para no
eliminarlos todos los valores mayores a 7 serán considerados como 7

In [69]:
print(df['passenger_count'].value_counts())

1    691848
2    147776
5     70898
3     43887
4     21225
6     20807
0      3554
9         1
Name: passenger_count, dtype: int64


In [70]:
df.loc[df.passenger_count > 7, 'passenger_count'] = 7

In [71]:
print(df['passenger_count'].value_counts())

1    691848
2    147776
5     70898
3     43887
4     21225
6     20807
0      3554
7         1
Name: passenger_count, dtype: int64


In [72]:
print(df.dtypes)
print(df.head())

key                          object
fare_amount                 float64
pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
passenger_count               int64
pickup_hour                   int64
dtype: object
                                   key  ...  pickup_hour
1000000    2010-03-06 21:13:04.0000003  ...           21
1000001  2010-03-10 16:19:00.000000128  ...           16
1000002    2014-02-27 08:50:19.0000006  ...            8
1000003    2011-02-11 17:46:17.0000005  ...           17
1000004   2011-07-25 12:48:00.00000074  ...           12

[5 rows x 9 columns]


In [73]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour
1000000,2010-03-06 21:13:04.0000003,10.1,2010-03-06 21:13:04,-73.993113,40.755552,-73.969351,40.797908,1,21
1000001,2010-03-10 16:19:00.000000128,3.7,2010-03-10 16:19:00,-73.948102,40.770608,-73.952923,40.768025,1,16
1000002,2014-02-27 08:50:19.0000006,11.0,2014-02-27 08:50:19,-73.977236,40.743045,-73.997854,40.719837,1,8
1000003,2011-02-11 17:46:17.0000005,7.3,2011-02-11 17:46:17,-73.954060,39.603285,-73.977015,40.776712,3,17
1000004,2011-07-25 12:48:00.00000074,6.9,2011-07-25 12:48:00,-73.978575,40.753067,-73.964632,40.764258,1,12


###Creando columna de distancia recorrida

Hallaremos la distancia recorrida en km para cada fila. \\
Usamos la Haversine formula, que halla la distancia entre dos puntos en una esfera dadas sus longitudes y latitudes. \\
$d = 2r \arcsin (\sqrt{\sin^{2}(\frac{\varphi_2 - \varphi_1}{2}) + \cos(\varphi_1)cos(\varphi_2)\sin^{2}(\frac{\lambda_2 - \lambda_1}{2})})$ \\
Sea $\varphi_1, \varphi_2$ latitudes y $\lambda_1, \lambda_2$ longitudes \\
referencia: https://en.wikipedia.org/wiki/Haversine_formula

Para disminuir los cálculos, lo llevaremos a ángulo doble: $\sin^{2}(\frac{\varphi_2 - \varphi_1}{2}) = \frac{1-\cos(\varphi_2 - \varphi_1)}{2}$

In [74]:
from math import cos, asin, sqrt, pi
p = pi/180
radioTierra = 6371
df["distancia_recorrida"] = df.apply(
    lambda fila: 2*radioTierra * asin(sqrt(0.5 - cos((fila.dropoff_latitude - fila.pickup_latitude) * p)/2 + cos(fila.pickup_latitude * p) * cos(fila.dropoff_latitude * p) * (1 - cos((fila.dropoff_longitude - fila.pickup_longitude) * p)) / 2)) 
, axis=1)

In [75]:
df.head(10)

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,distancia_recorrida
1000000,2010-03-06 21:13:04.0000003,10.1,2010-03-06 21:13:04,-73.993113,40.755552,-73.969351,40.797908,1,21,5.117160
1000001,2010-03-10 16:19:00.000000128,3.7,2010-03-10 16:19:00,-73.948102,40.770608,-73.952923,40.768025,1,16,0.497314
1000002,2014-02-27 08:50:19.0000006,11.0,2014-02-27 08:50:19,-73.977236,40.743045,-73.997854,40.719837,1,8,3.110906
1000003,2011-02-11 17:46:17.0000005,7.3,2011-02-11 17:46:17,-73.954060,39.603285,-73.977015,40.776712,3,17,130.493625
1000004,2011-07-25 12:48:00.00000074,6.9,2011-07-25 12:48:00,-73.978575,40.753067,-73.964632,40.764258,1,12,1.711032
1000005,2014-07-23 10:16:44.0000001,20.5,2014-07-23 10:16:44,-73.997789,40.741463,-73.984134,40.780437,1,10,4.483717
1000006,2015-03-27 20:55:48.0000003,16.5,2015-03-27 20:55:48,-73.938957,40.821960,-73.954590,40.768120,1,20,6.129730
1000007,2009-03-04 19:38:00.000000250,5.7,2009-03-04 19:38:00,-73.978618,40.777623,-73.987258,40.764442,1,19,1.636317
1000008,2014-01-13 16:42:00.000000119,21.5,2014-01-13 16:42:00,-73.972545,40.790757,-73.989142,40.740922,1,16,5.714965
1000009,2010-05-10 12:22:18.0000002,6.9,2010-05-10 12:22:18,-74.002299,40.745170,-73.994495,40.732993,1,12,1.505215


Eliminamos valores raros, donde la distancia no va acorde con la tarifa

In [76]:
import numpy as np
##Se crea una nueva columna con valores "ok" y "no"
df["booleano"]= np.where((df["distancia_recorrida"]>100)& (df["fare_amount"]<20),'no','ok')


In [77]:
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,distancia_recorrida,booleano
1000000,2010-03-06 21:13:04.0000003,10.1,2010-03-06 21:13:04,-73.993113,40.755552,-73.969351,40.797908,1,21,5.117160,ok
1000001,2010-03-10 16:19:00.000000128,3.7,2010-03-10 16:19:00,-73.948102,40.770608,-73.952923,40.768025,1,16,0.497314,ok
1000002,2014-02-27 08:50:19.0000006,11.0,2014-02-27 08:50:19,-73.977236,40.743045,-73.997854,40.719837,1,8,3.110906,ok
1000003,2011-02-11 17:46:17.0000005,7.3,2011-02-11 17:46:17,-73.954060,39.603285,-73.977015,40.776712,3,17,130.493625,no
1000004,2011-07-25 12:48:00.00000074,6.9,2011-07-25 12:48:00,-73.978575,40.753067,-73.964632,40.764258,1,12,1.711032,ok


In [78]:
#eliminamos todas las filas que tengan "no"
indices = df[df['booleano']=='no'].index
df.drop(indices, inplace=True)

In [79]:
#eliminamos la columna booleano
df.drop(columns=["booleano"], inplace=True)
df

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_hour,distancia_recorrida
1000000,2010-03-06 21:13:04.0000003,10.1,2010-03-06 21:13:04,-73.993113,40.755552,-73.969351,40.797908,1,21,5.117160
1000001,2010-03-10 16:19:00.000000128,3.7,2010-03-10 16:19:00,-73.948102,40.770608,-73.952923,40.768025,1,16,0.497314
1000002,2014-02-27 08:50:19.0000006,11.0,2014-02-27 08:50:19,-73.977236,40.743045,-73.997854,40.719837,1,8,3.110906
1000004,2011-07-25 12:48:00.00000074,6.9,2011-07-25 12:48:00,-73.978575,40.753067,-73.964632,40.764258,1,12,1.711032
1000005,2014-07-23 10:16:44.0000001,20.5,2014-07-23 10:16:44,-73.997789,40.741463,-73.984134,40.780437,1,10,4.483717
...,...,...,...,...,...,...,...,...,...,...
1999995,2014-11-17 13:51:02.0000002,4.0,2014-11-17 13:51:02,-73.950787,40.770836,-73.951043,40.774543,1,13,0.412763
1999996,2013-02-25 17:51:28.0000004,7.0,2013-02-25 17:51:28,-73.983035,40.748276,-73.974725,40.741978,1,17,0.990211
1999997,2013-07-12 22:03:41.0000006,10.5,2013-07-12 22:03:41,-74.003482,40.743577,-73.976758,40.765265,1,22,3.298915
1999998,2009-01-17 09:35:37.0000002,10.9,2009-01-17 09:35:37,-73.971819,40.797243,-73.985921,40.752459,3,9,5.119378


#Usando algoritmos y entrenando

Se usarán las siguientes carácteristicas


In [84]:
X = df[['passenger_count','pickup_hour','distancia_recorrida']]
y = df.fare_amount
print(X)

         passenger_count  pickup_hour  distancia_recorrida
1000000                1           21             5.117160
1000001                1           16             0.497314
1000002                1            8             3.110906
1000004                1           12             1.711032
1000005                1           10             4.483717
...                  ...          ...                  ...
1999995                1           13             0.412763
1999996                1           17             0.990211
1999997                1           22             3.298915
1999998                3            9             5.119378
1999999                2           15             5.705464

[998158 rows x 3 columns]


In [81]:
from sklearn.preprocessing import RobustScaler
transformer = RobustScaler(quantile_range = (0.1,0.9)).fit(X)
X = transformer.transform(X)

In [82]:
print(X)

[[ 0.          7.          3.00480136]
 [ 0.          2.         -1.61504472]
 [ 0.         -6.          0.99854716]
 ...
 [ 0.          8.          1.18655586]
 [ 2.         -5.          3.00701925]
 [ 1.          1.          3.5931046 ]]


In [83]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)